In [ ]:
#htsf\Scripts\activate    

# Import libraries
import pandas as pd
import numpy as np
#import matplotlib.pylab as plt ### Remove
#plt.style.use('bmh') ### Remove

import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'src'))
import hyperarch
from utils import get_sample_data, plot_single

%matplotlib inline

# Read data
h_df = get_sample_data(verbose = True)
g_df = get_sample_data(agg_type='grouped')

In [3]:
hierarchy_df, h_bottom, h_labels = hyperarch.get_hierarchal(h_df, 'category', 'subcategory', agg_type='hierarchy') 
hierarchy_df.head()

,total,category 1,category 2,category 1_subcat 1,category 1_subcat 2,category 1_subcat 3,category 2_subcat 4,category 2_subcat 5
date,,,,,,,,
2010-01-01,13.0,4.0,9.0,1.0,2.0,1.0,1.0,8.0
2010-02-01,12.1,3.1,9.0,2.0,1.0,0.1,3.0,6.0
2010-03-01,21.0,5.0,16.0,3.0,2.0,0.0,4.0,12.0
2010-04-01,24.5,9.5,15.0,3.3,5.0,1.2,3.0,12.0
2010-05-01,35.4,22.4,13.0,5.4,12.0,5.0,1.0,12.0


In [44]:
import pmdarima as pm
from pmdarima import model_selection
from sklearn.metrics import mean_squared_error

def hier_auto_arima(col, future=False, horizon=6, verbose=True, **kwargs):
    train, test = model_selection.train_test_split(col, **{'train_size': .8, **kwargs})
    # SARIMA(p, d, q)(P, D, Q)m 
    mod = pm.auto_arima(
        train, 
        **{
            'start_p' : 0, # p - Trend Order autoregression
            'start_q' : 0, # q - Trend Order 
            'start_P' : 0, # P - Seasonal Order autoregression
            'start_Q' : 0, # Q - Seasonal Order MA
            'max_p' : 5, 
            'max_q' : 5, 
            'max_P' : 3, 
            'max_Q' : 3, 
            'max_d' : 3, # d - Trend Order difference
            'max_D' : 3, # D - Seasonal Order Difference
            'm' : 12, # m - time steps in seasonal period
            'seasonal' : True, 
            'stepwise' : True, 
            'suppress_warnings' : True, 
            'error_action' : 'ignore',
            **kwargs
            }
        )
    test_preds, test_conf_int = mod.predict(n_periods=test.shape[0], return_conf_int=True)
    if verbose is True:
        print(f"Test RMSE - {col.name}: %.3f" % np.sqrt(mean_squared_error(test, test_preds)))
    if future is False:
        return mod, test_preds, test_conf_int
    else:
        preds, conf_int = mod.predict(n_periods=horizon, return_conf_int=True)
        return mod, preds, conf_int

m, p, c = hier_auto_arima(hierarchy_df.total, train_size=.75, future=True)

Test RMSE - total: 15.108


In [45]:
print(m.order)
print(m.seasonal_order)
p

(3, 0, 0)
(2, 1, 0, 12)


array([49.00204759, 66.1558327 , 83.83721169, 88.16957429, 81.42346936,
       63.09787102])

In [ ]:
# Hierarchy
hierarchy_df, h_bottom, h_labels = hyperarch.get_hierarchal(h_df, 'category', 'subcategory', agg_type='hierarchy')   
h_s = hyperarch.get_S(h_bottom, h_labels, agg_type='hierarchy')
h_models = hyperarch.get_models(hierarchy_df, order=(1, 2, 12), steps_out=6, make_stationary=False)
h_yhat = hyperarch.get_forecast_matrix(h_models)
h_reconciled_yhat = hyperarch.reconcile(h_yhat, h_s, method='ols')
hdf_pred, hdf_rec = hyperarch.predict_hier(hierarchy_df, h_yhat, h_reconciled_yhat, h_labels, h_models['index'])
plot_single(hdf_pred, hdf_rec)

In [ ]:
# Grouped
grouped_df, g_bottom, g_labels = hyperarch.get_hierarchal(g_df, 'category', 'subcategory', agg_type='grouped')   
g_s = hyperarch.get_S(g_bottom, g_labels, agg_type='grouped')
g_models = hyperarch.get_models(grouped_df, order=(1, 2, 12), steps_out=6, make_stationary=False)
g_yhat = hyperarch.get_forecast_matrix(g_models)
g_reconciled_yhat = hyperarch.reconcile(g_yhat, g_s, method='ols')
gdf_pred, gdf_rec = hyperarch.predict_hier(grouped_df, g_yhat, g_reconciled_yhat, g_labels, g_models['index'])
plot_single(gdf_pred, gdf_rec)